In [1]:
import random
import nacl.encoding
import nacl.hash

from utils import *
from ore import *
from bplustree import *
from node import *
from read_data import *
from dtree import *
from mpc import sim_mpc_n
from he import sim_he_n
from rf import *

In [3]:
rf = RandomForest(n_tree=20, max_f=0.1)
dataset = read_libsvm(name='madelon')
rf.fit(dataset)

Choose 50/500 features.
Start training rf..
Training rf 1/20
Training rf 2/20
Training rf 3/20
Training rf 4/20
Training rf 5/20
Training rf 6/20
Training rf 7/20
Training rf 8/20
Training rf 9/20
Training rf 10/20
Training rf 11/20
Training rf 12/20
Training rf 13/20
Training rf 14/20
Training rf 15/20
Training rf 16/20
Training rf 17/20
Training rf 18/20
Training rf 19/20
Training rf 20/20
End training rf.
Saving rf.
Saving rf done.


In [4]:
rf.score(dataset, [x[-1] for x in dataset])

0.5065

In [5]:
dt = CartTree()
dt.fit(dataset)

In [6]:
dt.score(dataset, [x[-1] for x in dataset])

1.0

In [2]:
def test_ore_encoding_ore():
    ore = ORE(key='123')
    A = sorted([random.randint(-434531, 43454) for _ in range(100)])
    B = [scale_val(x) for x in A]
    C = [ore.encode(x) for x in B]
    co = Coordinator()
    co.receive_keys(C[:50], C[50:])
    co.mp_map.update(co.do_map)
    d = co.mp_map.copy()

    for i in range(len(C)):
        for j in range(i):
            x, y = OREncoding(d[C[j].x]), OREncoding(d[C[i].x])
            a = ore.compare(C[j], C[i])
            b = ore.compare(x, y)
            if a != -1 or a != b:
                print(i, j)

In [3]:
def simulate():
    co = Coordinator()
    do = DataOwner()
    mp = ModelProvider()
    mp.train_model()

    key_0 = str(random.randint(0, 1000))
    do.ore.key = key_0
    mp.ore.key = key_0
    do_encs = do.get_encodings()
    mp_encs = mp.get_encodings()
    co.receive_keys(mp_arr=mp_encs, do_arr=do_encs)

    mp.receive_map(co.mp_map)
    do.receive_map(co.do_map)

    enc_dataset = []
    for sample in do.dataset:
        _s = sample[:]
        for i in range(len(_s)-1):
            _v = do.ore.encode(scale_val(_s[i])).x
            _s[i] = OREncoding(do.ore_map[_v])
        enc_dataset.append(_s)

    ys = [x[-1] for x in enc_dataset]
    pred = mp.model.predict(enc_dataset)
    print(f'ACC: {accuracy_score(pred, ys) * 100:.2f}%')

# simulate()

In [4]:
import time


co = Coordinator()
do = DataOwner()
mp = ModelProvider()
mp.train_model()

key_0 = str(random.randint(0, 1000))
do.ore.key = key_0
mp.ore.key = key_0
do_encs = do.get_encodings()
mp_encs = mp.get_encodings()

st = time.time()
tt = co.receive_keys(mp_arr=mp_encs, do_arr=do_encs)
tp = (time.time() - st) * 1000
print(f'{tp/len(mp.dataset):.3f}ms')

# mp.receive_map(co.mp_map)
# do.receive_map(co.do_map)
# do.dataset = read_rna()
# enc_dataset = []
# _ys = []
# _st = time.time()
# _i = 0
# for sample in do.dataset:
#     _s = sample[:]
#     _i += 1
#     for i in range(len(_s)-1):
#         _v = do.ore.encode(scale_val(_s[i])).x
#         _s[i] = OREncoding(do.ore_map[_v])
#     enc_dataset.append(_s)
#     if _i in _xs:
#         _ys.append((time.time()-_st)*1000)
#     if _i > _xs[-1]:
#         break
# ys = [x[-1] for x in enc_dataset]
# pred = mp.model.predict(enc_dataset)
# str(_ys)

MP model score: 87.28%


  0%|          | 0/1647 [00:00<?, ?it/s]

  0%|          | 0/1654 [00:00<?, ?it/s]

All data inserted into tree.
All data mapped to encoding.
0.020ms
